In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE42133"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE42133"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE42133.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE42133.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE42133.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Disrupted functional neworks in autism underlie early brain maldevelopment and provide accurate classification"
!Series_summary	"The disrupted genetic mechanisms underlying neural abnormalities in Autism Spectrum Disorder remain mostly unknown and speculative. No biological marker nor genetic signature is currently available to assist with early diagnosis."
!Series_summary	"We identified a blood-based gene expression signature relevant to the brain pathophysiology in autism. Also we identified genes that are differentially expressed in ASD subjects vs controls and gene modules that efficiently classify ASD and TD subjects."
!Series_overall_design	"Leukocyte gene expression levels were analysed in autistic and tipically develoing infants and toddlers with the purpose to identify gene expression signatures relevant to the autistic brain and to assist in the classification of subjects."
Sample Characteristics Dictionary:
{0: ['dx (diagnosis): ASD', '

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from leukocytes
is_gene_available = True

# 2.1 Data Availability
# Trait (diagnosis) is available in row 0
trait_row = 0

# Age is not available in the sample characteristics
age_row = None

# Gender appears to be constant (all male) as indicated in row 1
# Since all subjects are male, this is not a useful variable for our associative study
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    # Extract the value after the colon and trim whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: ASD = 1, Control = 0
    if value.lower() == "asd":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    # Not needed as age data is not available
    return None

def convert_gender(value):
    # Not needed as gender data is not variable (all male)
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if applicable)
if trait_row is not None:
    # Since clinical_data is not defined in the prompt, assuming it was loaded in a previous step
    # If not, we'd need to load it first
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Assuming clinical_data was loaded in a previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Warning: clinical_data not found. Make sure it was loaded in a previous step.")


Preview of selected clinical features:
{'GSM1033105': [1.0], 'GSM1033106': [1.0], 'GSM1033107': [1.0], 'GSM1033108': [1.0], 'GSM1033109': [0.0], 'GSM1033110': [0.0], 'GSM1033111': [1.0], 'GSM1033112': [0.0], 'GSM1033113': [1.0], 'GSM1033114': [0.0], 'GSM1033115': [1.0], 'GSM1033116': [1.0], 'GSM1033117': [1.0], 'GSM1033118': [1.0], 'GSM1033119': [0.0], 'GSM1033120': [0.0], 'GSM1033121': [0.0], 'GSM1033122': [1.0], 'GSM1033123': [0.0], 'GSM1033124': [0.0], 'GSM1033125': [1.0], 'GSM1033126': [1.0], 'GSM1033127': [0.0], 'GSM1033128': [1.0], 'GSM1033129': [1.0], 'GSM1033130': [1.0], 'GSM1033131': [0.0], 'GSM1033132': [0.0], 'GSM1033133': [0.0], 'GSM1033134': [0.0], 'GSM1033135': [0.0], 'GSM1033136': [0.0], 'GSM1033137': [1.0], 'GSM1033138': [0.0], 'GSM1033139': [1.0], 'GSM1033140': [0.0], 'GSM1033141': [0.0], 'GSM1033142': [0.0], 'GSM1033143': [0.0], 'GSM1033144': [0.0], 'GSM1033145': [0.0], 'GSM1033146': [0.0], 'GSM1033147': [0.0], 'GSM1033148': [1.0], 'GSM1033149': [1.0], 'GSM1033150': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols.
# These are microarray probe identifiers that need to be mapped to gene symbols.
# Illumina probes (ILMN_xxxxxxx) require mapping to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols in the gene annotation dataframe
# From the preview, we can see that 'ID' contains probe identifiers (ILMN_xxxxxxx) 
# and 'Symbol' contains gene symbols

# 2. Get the gene mapping dataframe using get_gene_mapping from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This function properly handles many-to-many relationships as required
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Preview the results to verify the mapping worked correctly
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene expression data after mapping:
Shape: (21464, 147)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. We need to first create the selected_clinical_df using clinical_data from Step 1
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the extracted clinical features
preview = preview_df(selected_clinical_df)
print("Preview of selected clinical features:")
print(preview)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and demographic features are severely biased
trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
if trait_type == 'binary':
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Remove biased demographic features
unbiased_linked_data = linked_data.copy()
if 'Age' in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
        
if 'Gender' in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data, 
    note="Dataset contains gene expression data from iPSC-derived neurons of ASD patients and unaffected siblings."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was determined to be not usable for analysis.")

Preview of selected clinical features:
{'GSM1033105': [1.0], 'GSM1033106': [1.0], 'GSM1033107': [1.0], 'GSM1033108': [1.0], 'GSM1033109': [0.0], 'GSM1033110': [0.0], 'GSM1033111': [1.0], 'GSM1033112': [0.0], 'GSM1033113': [1.0], 'GSM1033114': [0.0], 'GSM1033115': [1.0], 'GSM1033116': [1.0], 'GSM1033117': [1.0], 'GSM1033118': [1.0], 'GSM1033119': [0.0], 'GSM1033120': [0.0], 'GSM1033121': [0.0], 'GSM1033122': [1.0], 'GSM1033123': [0.0], 'GSM1033124': [0.0], 'GSM1033125': [1.0], 'GSM1033126': [1.0], 'GSM1033127': [0.0], 'GSM1033128': [1.0], 'GSM1033129': [1.0], 'GSM1033130': [1.0], 'GSM1033131': [0.0], 'GSM1033132': [0.0], 'GSM1033133': [0.0], 'GSM1033134': [0.0], 'GSM1033135': [0.0], 'GSM1033136': [0.0], 'GSM1033137': [1.0], 'GSM1033138': [0.0], 'GSM1033139': [1.0], 'GSM1033140': [0.0], 'GSM1033141': [0.0], 'GSM1033142': [0.0], 'GSM1033143': [0.0], 'GSM1033144': [0.0], 'GSM1033145': [0.0], 'GSM1033146': [0.0], 'GSM1033147': [0.0], 'GSM1033148': [1.0], 'GSM1033149': [1.0], 'GSM1033150': [

Normalized gene data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE42133.csv


For the feature 'Autism_spectrum_disorder_(ASD)', the least common label is '0.0' with 56 occurrences. This represents 38.10% of the dataset.


Linked data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE42133.csv
